
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Apache_Parquet_logo.svg" width=400 alt="Parquet Logo"></img>

# Store Kerchunk Reference Files as Parquet

## Overview
   
In this notebook we will cover how to store Kerchunk references as Parquet files instead of json. For large reference datasets, using Parquet should have performance implications as the overall reference file size should be smaller and the memory overhead of combining the reference files should be lower. 


This notebook builds upon the [Kerchunk Basics](notebooks/foundations/01_kerchunk_basics.ipynb), [Multi-File Datasets with Kerchunk](notebooks/foundations/02_kerchunk_multi_file.ipynb) and the [Kerchunk and Dask](notebooks/foundations/03_kerchunk_dask.ipynb) notebooks. 

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Kerchunk Basics](../foundations/kerchunk_basics) | Required | Core |
| [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file) | Required | Core |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Recommended | IO/Visualization |
| [Intro to Dask](https://tutorial.dask.org/00_overview.html) | Required | Parallel Processing |

- **Time to learn**: 30 minutes
---

## Imports
In addition to the previous imports we used throughout the tutorial, we are adding a few imports:
- `LazyReferenceMapper` and `ReferenceFileSystem` from `fsspec.implementations.reference` for lazy Parquet.




In [ ]:
import logging
import os

import dask
import fsspec
import xarray as xr
from distributed import Client
from fsspec.implementations.reference import LazyReferenceMapper, ReferenceFileSystem
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

## Setting up the `Dask` Client


In [ ]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

## Create Input File List

Here we are using `fsspec's` glob functionality along with the *`*`* wildcard operator and some string slicing to grab a list of NetCDF files from a `s3` `fsspec` filesystem. 

In [ ]:
# Initiate fsspec filesystems for reading
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

files_paths = fs_read.glob("s3://smn-ar-wrf/DATA/WRF/DET/2022/12/31/12/*")

# Here we prepend the prefix 's3://', which points to AWS.
file_pattern = sorted(["s3://" + f for f in files_paths])

# Grab the first seven files to speed up example.
file_pattern = file_pattern[0:2]

# Generate Lazy References
Below we will create a `fsspec` filesystem to read the references from `s3` and create a function to generate dask delayed tasks.

In [ ]:
# Use Kerchunk's `SingleHdf5ToZarr` method to create a `Kerchunk`
# index from a NetCDF file.
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")


def generate_json_reference(fil):
    with fs_read.open(fil, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, fil, inline_threshold=300)
        return h5chunks.translate()  # outf


# Generate Dask Delayed objects
tasks = [dask.delayed(generate_json_reference)(fil) for fil in file_pattern]

### Start the Dask Processing
To view the processing you can view it in real-time on the Dask Dashboard. ex: http://127.0.0.1:8787/status

In [ ]:
single_refs = dask.compute(tasks)[0]

In [ ]:
len(single_refs)

## Combine In-Memory References with MultiZarrToZarr
This section will look notably different than the previous examples that have written to `.json`.

In the following code block we are:
- Creating an `fsspec` filesystem.
- Create a empty `parquet` file to write to. 
- Creating an `fsspec` `LazyReferenceMapper` to pass into `MultiZarrToZarr`
- Building a `MultiZarrToZarr` object of the combined references.
- Calling `.flush()` on our LazyReferenceMapper, to write the combined reference to our `parquet` file.

In [ ]:
fs = fsspec.filesystem("file")

if os.path.exists("combined.parq"):
    import shutil

    shutil.rmtree("combined.parq")
os.makedirs("combined.parq")

out = LazyReferenceMapper.create(root="combined.parq", fs=fs)

mzz = MultiZarrToZarr(
    single_refs,
    remote_protocol="memory",
    concat_dims=["time"],
    identical_dims=["y", "x"],
    out=out,
).translate()

out.flush()

# Shutdown the Dask cluster

In [ ]:
client.shutdown()

## Load kerchunked dataset

Next we initiate a `fsspec` `ReferenceFileSystem`.
We need to pass:
- The name of the parquet store
- The remote protocol (This is the protocol of the input file urls)
- The target protocol (`file` since we saved our parquet store locally).

In [ ]:
fs = ReferenceFileSystem(
    "combined.parq", remote_protocol="s3", target_protocol="file", lazy=True
)
ds = xr.open_dataset(
    fs.get_mapper(), engine="zarr", backend_kwargs={"consolidated": False}
)

In [ ]:
ds